# Traffic Sign Classification with Keras

Keras exists to make coding deep neural networks simpler. To demonstrate just how easy it is, you’re going to use Keras to build a convolutional neural network in a few dozen lines of code.

You’ll be connecting the concepts from the previous lessons to the methods that Keras provides.

## Dataset

The network you'll build with Keras is similar to the example that you can find in Keras’s GitHub repository that builds out a [convolutional neural network for MNIST](https://github.com/fchollet/keras/blob/master/examples/mnist_cnn.py). 

However, instead of using the [MNIST](http://yann.lecun.com/exdb/mnist/) dataset, you're going to use the [German Traffic Sign Recognition Benchmark](http://benchmark.ini.rub.de/?section=gtsrb&subsection=news) dataset that you've used previously.

You can download pickle files with sanitized traffic sign data here.

## Overview

Here are the steps you'll take to build the network:

1. First load the data.
2. Build a feedforward neural network to classify traffic signs.
3. Build a convolutional neural network to classify traffic signs.

Keep an eye on the network’s accuracy over time. Once the accuracy reaches the 98% range, you can be confident that you’ve built and trained an effective model.

## Load the Data

Start by importing the data from the pickle file.

In [209]:
# TODO: Implement load the data here.
import pickle
train_file = "train.p"
# test_file = ".test.p

with open(train_file, 'rb') as f:
    data_dict = pickle.load(f)
    X_train = data_dict['features']
    y_train = data_dict['labels']
    
# STOP: Do not change the tests below. Your implementation should pass these tests. 
assert(X_train.shape[0] == y_train.shape[0]), "The number of images is not equal to the number of labels."
assert(X_train.shape[1:] == (32,32,3)), "The dimensions of the images are not 32 x 32 x 3."

In [1]:
from keras.datasets import cifar10
(X_train, y_train), (X_test, y_test) = cifar10.load_data()

Using TensorFlow backend.


170491904/170498071 [============================>.] - ETA: 0sUntaring file...


## Normalize the data

Now that you've loaded the training data, normalize the input so that it has a mean of 0 and a range between -0.5 and 0.5.

In [2]:
from scipy import stats
import numpy as np
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Convolution2D, MaxPooling2D
from keras.optimizers import SGD
from keras.utils import np_utils

from sklearn.model_selection import train_test_split

In [3]:
# TODO: Implement data normalization here.
def center_scale(arr):
    xmax, xmin = arr.max(), arr.min()
    arr = ((arr - xmin) / (xmax - xmin)) -.5
    return arr

X_train = center_scale(X_train.astype(float))

# STOP: Do not change the tests below. Your implementation should pass these tests. 
assert(round(np.mean(X_train)) == 0), "The mean of the input data is: %f" % np.mean(X_train)
assert(np.min(X_train) == -0.5 and np.max(X_train) == 0.5), "The range of the input data is: %.1f to %.1f" % (np.min(X_train), np.max(X_train))

## Build a Two-Layer Feedfoward Network

The code you've written so far is for data processing, not specific to Keras. Here you're going to build Keras-specific code.

Build a two-layer feedforward neural network, with 128 neurons in the fully-connected hidden layer. 

To get started, review the Keras documentation about [models](https://keras.io/models/sequential/) and [layers](https://keras.io/layers/core/).

The Keras example of a [Multi-Layer Perceptron](https://github.com/fchollet/keras/blob/master/examples/mnist_mlp.py) network is similar to what you need to do here. Use that as a guide, but keep in mind that there are a number of differences.

In [203]:
# TODO: Build a two-layer feedforward neural network with Keras here.

# create model
model = Sequential()

# add 2 layers, feed forward, and 128 neurons in the fc
model.add(Dense(128, input_dim=32*32*3, init='uniform', name='hidden1'))
model.add(Activation('tanh'))
model.add(Dense(43, init='uniform', name='output'))
model.add(Activation('softmax'))

# STOP: Do not change the tests below. Your implementation should pass these tests.
assert(model.get_layer(name="hidden1").input_shape == (None, 32*32*3)), "The input shape is: %s" % model.get_layer(name="hidden1").input_shape
assert(model.get_layer(name="output").output_shape == (None, 43)), "The output shape is: %s" % model.get_layer(name="output").output_shape 

## Train the Network
Compile and train the network for 2 epochs. [Use the `adam` optimizer, with `categorical_crossentropy` loss.](https://keras.io/models/sequential/)

Hint 1: In order to use categorical cross entropy, you will need to [one-hot encode the labels](https://github.com/fchollet/keras/blob/master/keras/utils/np_utils.py).

Hint 2: In order to pass the input images to the fully-connected hidden layer, you will need to [reshape the input](https://github.com/fchollet/keras/blob/master/examples/mnist_mlp.py).

Hint 3: Keras's `.fit()` method returns a `History.history` object, which the tests below use. Save that to a variable named `history`.

In [204]:
# TODO: Compile and train the model here.

# compile
model.compile(loss='categorical_crossentropy', 
              optimizer='adam',
              metrics=['accuracy'])

# fit
history = model.fit(X_train.reshape(-1,32*32*3), np_utils.to_categorical(y_train), batch_size=64, nb_epoch=2)

# STOP: Do not change the tests below. Your implementation should pass these tests.
assert(history.history['acc'][-1] > 0.5), "The training accuracy was: %.3f" % history.history['acc']

Epoch 1/2
39209/39209 [==============================] - 4s - loss: 1.4680 - acc: 0.6252     
Epoch 2/2
39209/39209 [==============================] - 3s - loss: 0.6067 - acc: 0.8475     


## Validate the Network
Split the training data into a training and validation set.

Measure the [validation accuracy](https://keras.io/models/sequential/) of the network after two training epochs.

Hint: [Use the `train_test_split()` method](http://scikit-learn.org/stable/modules/generated/sklearn.model_selection.train_test_split.html) from scikit-learn.

In [205]:
# TODO: Split some of the training data into a validation dataset.


#transform the data 
X_train = X_train.reshape(-1, 32*32*3)
y_train = np_utils.to_categorical(y_train)

# split the dataset
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=.25, random_state=42)

# TODO: Compile and train the model to measure validation accuracy.
# compile
model.compile(loss='categorical_crossentropy', 
              optimizer='adam',
              metrics=['accuracy'])

# evaluate
#score = model.evaluate(X_val.reshape(-1, 32*32*3), np_utils.to_categorical(y_val), batch_size=64)
history = model.fit(X_train, y_train, batch_size=64, nb_epoch=2, validation_data=(X_val, y_val))

# STOP: Do not change the tests below. Your implementation should pass these tests.
assert(round(X_train.shape[0] / float(X_val.shape[0])) == 3), "The training set is %.3f times larger than the validation set." % X_train.shape[0] / float(X_val.shape[0])
assert(history.history['val_acc'][-1] > 0.6), "The validation accuracy is: %.3f" % history.history['val_acc'][0]

Train on 29406 samples, validate on 9803 samples
Epoch 1/2
29406/29406 [==============================] - 3s - loss: 0.4349 - acc: 0.8912 - val_loss: 0.3940 - val_acc: 0.8955
Epoch 2/2
29406/29406 [==============================] - 3s - loss: 0.3726 - acc: 0.9042 - val_loss: 0.3767 - val_acc: 0.8977


**Validation Accuracy**: 89.77%

## Congratulations
You've built a feedforward neural network in Keras!

Don't stop here! Next, you'll add a convolutional layer to drive.py.

## Convolutions
Build a new network, similar to your existing network. Before the hidden layer, add a 3x3 [convolutional layer](https://keras.io/layers/convolutional/#convolution2d) with 32 filters and valid padding.

Then compile and train the network.

Hint 1: The Keras example of a [convolutional neural network](https://github.com/fchollet/keras/blob/master/examples/mnist_cnn.py) for MNIST would be a good example to review.

Hint 2: Now that the first layer of the network is a convolutional layer, you no longer need to reshape the input images before passing them to the network. You might need to reload your training data to recover the original shape.

Hint 3: Add a [`Flatten()` layer](https://keras.io/layers/core/#flatten) between the convolutional layer and the fully-connected hidden layer.

In [215]:
X_train.shape

(39209, 32, 32, 3)

In [5]:
# split the dataset
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=.25, random_state=42)

In [218]:
# TODO: Re-construct the network and add a convolutional layer before the first fully-connected layer.

# create model
model = Sequential()

# add 2 layers, feed forward, and 128 neurons in the fc
model.add(Convolution2D(32, 3, 3 ,border_mode='valid',
                        input_shape=(32, 32, 3), init='uniform', name='conv1'))
model.add(Activation('tanh'))
model.add(Flatten())
model.add(Dense(128, init='uniform', name='hidden1'))
model.add(Activation('tanh'))
model.add(Dense(43, init='uniform', name='output'))
model.add(Activation('softmax'))

# compile
model.compile(loss='categorical_crossentropy', 
              optimizer='adam',
              metrics=['accuracy'])

# fit
history = model.fit(X_train, np_utils.to_categorical(y_train), batch_size=64, nb_epoch=2, 
                    validation_data=(X_val, np_utils.to_categorical(y_val)))

# STOP: Do not change the tests below. Your implementation should pass these tests.
assert(history.history['val_acc'][-1] > 0.9), "The validation accuracy is: %.3f" % history.history['val_acc'][0]

Train on 22054 samples, validate on 7352 samples
Epoch 1/2
22054/22054 [==============================] - 31s - loss: 1.1920 - acc: 0.7191 - val_loss: 0.3949 - val_acc: 0.9215
Epoch 2/2
22054/22054 [==============================] - 31s - loss: 0.2648 - acc: 0.9457 - val_loss: 0.2460 - val_acc: 0.9498


**Validation Accuracy**: 94.98%

## Pooling
Re-construct your network and add a 2x2 [pooling layer](https://keras.io/layers/pooling/#maxpooling2d) immediately following your convolutional layer.

Then compile and train the network.

In [223]:
# already splitted betwen train and test

# create model
model = Sequential()

# add 2 layers, feed forward, and 128 neurons in the fc
model.add(Convolution2D(32, 3, 3 ,border_mode='valid',
                        input_shape=(32, 32, 3), init='uniform', name='conv1'))
model.add(Activation('tanh'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Flatten())
model.add(Dense(128, init='uniform', name='hidden1'))
model.add(Activation('tanh'))
model.add(Dense(43, init='uniform', name='output'))
model.add(Activation('softmax'))

# compile
model.compile(loss='categorical_crossentropy', 
              optimizer='adam',
              metrics=['accuracy'])

# fit
history = model.fit(X_train, np_utils.to_categorical(y_train), batch_size=64, nb_epoch=2, 
                    validation_data=(X_val, np_utils.to_categorical(y_val)))

# STOP: Do not change the tests below. Your implementation should pass these tests.
assert(history.history['val_acc'][-1] > 0.9), "The validation accuracy is: %.3f" % history.history['val_acc'][0]

Train on 22054 samples, validate on 7352 samples
Epoch 1/2
22054/22054 [==============================] - 17s - loss: 1.7098 - acc: 0.5508 - val_loss: 0.6785 - val_acc: 0.8383
Epoch 2/2
22054/22054 [==============================] - 17s - loss: 0.3975 - acc: 0.9209 - val_loss: 0.2577 - val_acc: 0.9495


**Validation Accuracy**: 94.95%

## Dropout
Re-construct your network and add [dropout](https://keras.io/layers/core/#dropout) after the pooling layer. Set the dropout rate to 50%.

In [224]:
# already splitted betwen train and test

# create model
model = Sequential()

# add 2 layers, feed forward, and 128 neurons in the fc
model.add(Convolution2D(32, 3, 3 ,border_mode='valid',
                        input_shape=(32, 32, 3), init='uniform', name='conv1'))
model.add(Activation('tanh'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(.50))
model.add(Flatten())
model.add(Dense(128, init='uniform', name='hidden1'))
model.add(Activation('tanh'))
model.add(Dense(43, init='uniform', name='output'))
model.add(Activation('softmax'))

# compile
model.compile(loss='categorical_crossentropy', 
              optimizer='adam',
              metrics=['accuracy'])

# fit
history = model.fit(X_train, np_utils.to_categorical(y_train), batch_size=64, nb_epoch=2, 
                    validation_data=(X_val, np_utils.to_categorical(y_val)))
# STOP: Do not change the tests below. Your implementation should pass these tests.
assert(history.history['val_acc'][-1] > 0.9), "The validation accuracy is: %.3f" % history.history['val_acc'][0]

Train on 22054 samples, validate on 7352 samples
Epoch 1/2
22054/22054 [==============================] - 19s - loss: 1.8047 - acc: 0.5230 - val_loss: 0.7748 - val_acc: 0.8035
Epoch 2/2
22054/22054 [==============================] - 18s - loss: 0.5222 - acc: 0.8800 - val_loss: 0.3036 - val_acc: 0.9332


**Validation Accuracy**: 93.32%

## Optimization
Congratulations! You've built a neural network with convolutions, pooling, dropout, and fully-connected layers, all in just a few lines of code.

Have fun with the model and see how well you can do! Add more layers, or regularization, or different padding, or batches, or more training epochs.

What is the best validation accuracy you can achieve?

In [7]:
# already splitted betwen train and test

# create model
model = Sequential()

# add 2 layers, feed forward, and 128 neurons in the fc

# 1
model.add(Convolution2D(100, 7, 7 ,border_mode='valid',
                        input_shape=(32, 32, 3), init='uniform', name='conv1'))
model.add(Activation('tanh'))
model.add(MaxPooling2D(pool_size=(2,2), border_mode='valid'))

# 2
model.add(Convolution2D(150, 4, 4 ,border_mode='valid',
                        input_shape=(32, 32, 3), init='uniform', name='conv2'))
model.add(Activation('tanh'))
model.add(MaxPooling2D(pool_size=(2,2), border_mode='valid'))

# 3
model.add(Convolution2D(250, 4, 4 ,border_mode='valid',
                        input_shape=(32, 32, 3), init='uniform', name='conv3'))
model.add(Activation('tanh'))

#4
model.add(Flatten())
model.add(Dense(300, init='uniform', name='fc'))
model.add(Activation('tanh'))
model.add(Dense(10, init='uniform', name='output'))
model.add(Activation('softmax'))

# compile
model.compile(loss='categorical_crossentropy', 
              optimizer='adam',
              metrics=['accuracy'])

# fit
history = model.fit(X_train, np_utils.to_categorical(y_train), batch_size=128, nb_epoch=2, 
                    validation_data=(X_val, np_utils.to_categorical(y_val)))
# STOP: Do not change the tests below. Your implementation should pass these tests.
assert(history.history['val_acc'][-1] > 0.9), "The validation accuracy is: %.3f" % history.history['val_acc'][0]

Train on 37500 samples, validate on 12500 samples
Epoch 1/2
37500/37500 [==============================] - 200s - loss: 1.5584 - acc: 0.4387 - val_loss: 1.3412 - val_acc: 0.5205
Epoch 2/2
37500/37500 [==============================] - 199s - loss: 1.2279 - acc: 0.5615 - val_loss: 1.1680 - val_acc: 0.5810


AssertionError: The validation accuracy is: 0.520

**Best Validation Accuracy:** 98.20%

## Testing
Once you've picked out your best model, it's time to test it.

Load up the test data and use the [`evaluate()` method](https://keras.io/models/model/#evaluate) to see how well it does.

Hint 1: After you load your test data, don't forget to normalize the input and one-hot encode the output, so it matches the training data.

Hint 2: The `evaluate()` method should return an array of numbers. Use the `metrics_names()` method to get the labels.

In [228]:
with open('./test.p', mode='rb') as f:
    test = pickle.load(f)
    
X_test = test['features']
y_test = test['labels']
X_test = X_test.astype('float32')
X_test /= 255
X_test -= 0.5
Y_test = np_utils.to_categorical(y_test, 43)

scores = model.evaluate(X_test, Y_test)

12630/12630 [==============================] - 18s    


In [229]:
scores # test loss and accuracy

[0.19747993702630079, 0.95574030088981932]

**Test Accuracy:** 95.57%

In [242]:
# other tests
img = X_train[0]
a = np.zeros((*img.shape,3), dtype=np.uint8)
a.shape

(32, 32, 3, 3)

## Summary
Keras is a great tool to use if you want to quickly build a neural network and evaluate performance.